In [2]:
from keras.datasets import mnist
from keras.engine.topology import Layer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, TimeDistributed
from keras.utils import to_categorical
from keras import optimizers

import t3f
import numpy as np

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.layers import Recurrent

# First, let's define a RNN Cell, as a layer subclass.

class TTMinimalRNNCell(Recurrent):
    counter = 0

    def __init__(self, row_dims, column_dims, tt_rank=2, init='glorot',
                 activation='relu', bias=True, bias_init=0.1, **kwargs):
        self.units = np.prod(column_dims)
        self.states = [None]
        
        self.tt_kernel_shape = [row_dims, column_dims]
        self.tt_recurrent_shape = [column_dims, column_dims]
        self.output_dim = np.prod(column_dims)
        self.tt_rank = tt_rank
        self.activation = activation
        self.bias = bias
        self.bias_init = bias_init
        self.init = init
        
        super(TTMinimalRNNCell, self).__init__(**kwargs)        

    def build(self, input_shape):
        # kernel
        initializer = t3f.glorot_initializer(self.tt_kernel_shape,
                                        tt_rank=self.tt_rank)
        name = 'tt_rnn_kernel_matrix_{}'.format(TTMinimalRNNCell.counter)
        self.kernel = t3f.get_variable(name, initializer=initializer)
        
        
        # recurrent
        initializer = t3f.glorot_initializer(self.tt_recurrent_shape,
                                        tt_rank=self.tt_rank)
        name = 'tt_rnn_recurrent_matrix_{}'.format(TTMinimalRNNCell.counter)
        self.recurrent_kernel = t3f.get_variable(name, initializer=initializer)
        
        self.trainable_weights = list(self.kernel.tt_cores) + list(self.recurrent_kernel.tt_cores)
        
        TTMinimalRNNCell.counter += 1
        self.built = True

    def step(self, inputs, states):
        prev_output = states[0]
        h = t3f.matmul(inputs, self.kernel)
        z = t3f.matmul(prev_output, self.recurrent_kernel)
        
        output = h + z
        return output, [output]

In [14]:
model = Sequential()
model.add(TimeDistributed(Flatten(), input_shape=(28, 28)))
model.add(TTMinimalRNNCell(row_dims=[4, 7], column_dims=[5, 5], tt_rank=4))
model.add(Dense(10))
model.add(Activation('softmax'))

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_2 (TimeDist (None, 28, 28)            0         
_________________________________________________________________
tt_minimal_rnn_cell_2 (TTMin (None, 25)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                260       
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
Total params: 680
Trainable params: 680
Non-trainable params: 0
_________________________________________________________________


In [16]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 127.5 - 1.0
x_test = x_test / 127.5 - 1.0

y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [17]:
optimizer = optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
model.fit(x_train, y_train, epochs=100, batch_size=64, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 55s - loss: 3.1140 - acc: 0.2363 - val_loss: 1.7060 - val_acc: 0.3764
Epoch 2/100
60000/60000 [==============================] - 53s - loss: 1.3132 - acc: 0.5536 - val_loss: 1.0077 - val_acc: 0.6810
Epoch 3/100
60000/60000 [==============================] - 55s - loss: 0.9502 - acc: 0.7035 - val_loss: 0.8565 - val_acc: 0.7312
Epoch 4/100
60000/60000 [==============================] - 54s - loss: 0.8632 - acc: 0.7363 - val_loss: 0.8067 - val_acc: 0.7558
Epoch 5/100
60000/60000 [==============================] - 54s - loss: 0.8112 - acc: 0.7544 - val_loss: 0.7702 - val_acc: 0.7603
Epoch 6/100
60000/60000 [==============================] - 53s - loss: 0.7813 - acc: 0.7618 - val_loss: 0.7539 - val_acc: 0.7768
Epoch 7/100
60000/60000 [==============================] - 50s - loss: 0.7617 - acc: 0.7687 - val_loss: 0.7417 - val_acc: 0.7735
Epoch 8/100
60000/60000 [======================

60000/60000 [==============================] - 50s - loss: 0.4802 - acc: 0.8619 - val_loss: 0.4724 - val_acc: 0.8649
Epoch 64/100
60000/60000 [==============================] - 51s - loss: 0.4787 - acc: 0.8621 - val_loss: 0.4714 - val_acc: 0.8653
Epoch 65/100
60000/60000 [==============================] - 51s - loss: 0.4795 - acc: 0.8619 - val_loss: 0.4692 - val_acc: 0.8634
Epoch 66/100
60000/60000 [==============================] - 50s - loss: 0.4784 - acc: 0.8626 - val_loss: 0.4626 - val_acc: 0.8664
Epoch 67/100
60000/60000 [==============================] - 51s - loss: 0.4754 - acc: 0.8620 - val_loss: 0.4633 - val_acc: 0.8677
Epoch 68/100
60000/60000 [==============================] - 52s - loss: 0.4745 - acc: 0.8630 - val_loss: 0.4747 - val_acc: 0.8648
Epoch 69/100
60000/60000 [==============================] - 50s - loss: 0.4755 - acc: 0.8637 - val_loss: 0.4587 - val_acc: 0.8676
Epoch 70/100
60000/60000 [==============================] - 60s - loss: 0.4739 - acc: 0.8641 - val_loss